In [1]:
from pathlib import Path

import pandas as pd
from lightning import pytorch as pl
import ray
from ray import tune
from ray.train import CheckpointConfig, RunConfig, ScalingConfig
from ray.train.lightning import (RayDDPStrategy, RayLightningEnvironment,
                                 RayTrainReportCallback, prepare_trainer)
from ray.train.torch import TorchTrainer
from ray.tune.search.hyperopt import HyperOptSearch
from ray.tune.search.optuna import OptunaSearch
from ray.tune.schedulers import FIFOScheduler

from chemprop import data, featurizers, models, nn

/home/aih/serra.korkmaz/miniconda3/envs/saturn/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-12 00:31:21,644	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-03-12 00:31:21,944	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-03-12 00:31:22,362	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
chemprop_dir = Path.cwd().parent
train_path = "/home/aih/serra.korkmaz/projects/saturn/SurogateModel/training_data_4sm_zinc/train_data_fa7.csv"
test_path = "/home/aih/serra.korkmaz/projects/saturn/SurogateModel/training_data_4sm_zinc/test_data_fa7.csv"
num_workers = 0 # number of workers for dataloader. 0 means using main process for data loading
smiles_column = 'smiles' # name of the column containing SMILES strings
target_columns = ['value'] # list of names of the columns containing targets

hpopt_save_dir = Path.cwd() / "hpopt_fa7_zinc" # directory to save hyperopt results
hpopt_save_dir.mkdir(exist_ok=True)

In [3]:
df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)

In [4]:
# Extract SMILES and target values
train_smis = df_train[smiles_column].values
train_ys = df_train[target_columns].values

test_smis = df_test[smiles_column].values
test_ys = df_test[target_columns].values

# Convert data to MoleculeDatapoint format
train_data = [data.MoleculeDatapoint.from_smi(smi, y) for smi, y in zip(train_smis, train_ys)]
test_data = [data.MoleculeDatapoint.from_smi(smi, y) for smi, y in zip(test_smis, test_ys)]


In [5]:
# Initialize featurizer
featurizer = featurizers.SimpleMoleculeMolGraphFeaturizer()

# Create training dataset
train_dset = data.MoleculeDataset(train_data, featurizer)
scaler = train_dset.normalize_targets()

# Split validation set from training data (80% train, 20% validation)
split_idx = int(len(train_data) * 0.8)
val_data = train_data[split_idx:]
train_data = train_data[:split_idx]

# Create validation and test datasets
val_dset = data.MoleculeDataset(val_data, featurizer)
val_dset.normalize_targets(scaler)

test_dset = data.MoleculeDataset(test_data, featurizer)

In [6]:
def train_model(config, train_dset, val_dset, num_workers, scaler):

    # config is a dictionary containing hyperparameters used for the trial
    depth = int(config["depth"])
    ffn_hidden_dim = int(config["ffn_hidden_dim"])
    ffn_num_layers = int(config["ffn_num_layers"])
    message_hidden_dim = int(config["message_hidden_dim"])

    train_loader = data.build_dataloader(train_dset, num_workers=num_workers, shuffle=True)
    val_loader = data.build_dataloader(val_dset, num_workers=num_workers, shuffle=False)

    mp = nn.BondMessagePassing(d_h=message_hidden_dim, depth=depth)
    agg = nn.MeanAggregation()
    output_transform = nn.UnscaleTransform.from_standard_scaler(scaler)
    ffn = nn.RegressionFFN(output_transform=output_transform, input_dim=message_hidden_dim, hidden_dim=ffn_hidden_dim, n_layers=ffn_num_layers)
    batch_norm = True
    metric_list = [nn.metrics.RMSE(), nn.metrics.MAE()]
    model = models.MPNN(mp, agg, ffn, batch_norm, metric_list)

    trainer = pl.Trainer(
        accelerator="auto",
        devices=1,
        max_epochs=20, # number of epochs to train for
        # below are needed for Ray and Lightning integration
        strategy=RayDDPStrategy(),
        callbacks=[RayTrainReportCallback()],
        plugins=[RayLightningEnvironment()],
    )

    trainer = prepare_trainer(trainer)
    trainer.fit(model, train_loader, val_loader)

In [7]:
search_space = {
    "depth": tune.qrandint(lower=2, upper=6, q=1),
    "ffn_hidden_dim": tune.qrandint(lower=300, upper=2400, q=100),
    "ffn_num_layers": tune.qrandint(lower=1, upper=3, q=1),
    "message_hidden_dim": tune.qrandint(lower=300, upper=2400, q=100),
}

In [8]:
ray.init()

scheduler = FIFOScheduler()

# Scaling config controls the resources used by Ray
scaling_config = ScalingConfig(
    num_workers=1,
    use_gpu=False, # change to True if you want to use GPU
)

# Checkpoint config controls the checkpointing behavior of Ray
checkpoint_config = CheckpointConfig(
    num_to_keep=1, # number of checkpoints to keep
    checkpoint_score_attribute="val_loss", # Save the checkpoint based on this metric
    checkpoint_score_order="min", # Save the checkpoint with the lowest metric value
)

run_config = RunConfig(
    checkpoint_config=checkpoint_config,
    storage_path=hpopt_save_dir / "ray_results", # directory to save the results
)

ray_trainer = TorchTrainer(
    lambda config: train_model(
        config, train_dset, val_dset, num_workers, scaler
    ),
    scaling_config=scaling_config,
    run_config=run_config,
)

search_alg = HyperOptSearch(
    n_initial_points=1, # number of random evaluations before tree parzen estimators
    random_state_seed=42,
)

# OptunaSearch is another search algorithm that can be used
# search_alg = OptunaSearch()

tune_config = tune.TuneConfig(
    metric="val_loss",
    mode="min",
    num_samples=2, # number of trials to run
    scheduler=scheduler,
    search_alg=search_alg,
    trial_dirname_creator=lambda trial: str(trial.trial_id), # shorten filepaths

)

tuner = tune.Tuner(
    ray_trainer,
    param_space={
        "train_loop_config": search_space,
    },
    tune_config=tune_config,
)

# Start the hyperparameter search
results = tuner.fit()

(TorchTrainer pid=2734610) Started distributed worker processes: 
(TorchTrainer pid=2734610) - (node_id=8ef1e3bc9f521069ce86078c551759efba54d44695801db33cd4ef07, ip=10.233.0.55, pid=2734849) world_rank=0, local_rank=0, node_rank=0
(RayTrainWorker pid=2734849) Setting up process group for: env:// [rank=0, world_size=1]
(RayTrainWorker pid=2734849) /home/aih/serra.korkmaz/miniconda3/envs/saturn/lib/python3.11/site-packages/ray/train/lightning/_lightning_utils.py:262: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
(RayTrainWorker pid=2734849) `get_trial_name` is deprecated because the concept of a `Trial` will soon be removed in Ray Train.Ray Train will no longer assume that it's running within a Ray Tune `Trial` in the future. See this issue for more context and migration options: https://github.com/ray-project/ray/issues/49454. Disable these warnings by setting the envi

Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]


(RayTrainWorker pid=2734849) /home/aih/serra.korkmaz/miniconda3/envs/saturn/lib/python3.11/site-packages/lightning/pytorch/core/saving.py:363: Skipping 'metrics' parameter because it is not possible to safely dump to YAML.
(RayTrainWorker pid=2734849) /home/aih/serra.korkmaz/miniconda3/envs/saturn/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.


Epoch 0:  68%|██████▊   | 19/28 [00:07<00:03,  2.57it/s, v_num=3.41e+7, train_loss_step=0.469]


(TorchTrainer pid=2734850) Started distributed worker processes: 
(TorchTrainer pid=2734850) - (node_id=8ef1e3bc9f521069ce86078c551759efba54d44695801db33cd4ef07, ip=10.233.0.55, pid=2735082) world_rank=0, local_rank=0, node_rank=0
(RayTrainWorker pid=2735082) Setting up process group for: env:// [rank=0, world_size=1]


...

(RayTrainWorker pid=2734849) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/ictstr01/home/aih/serra.korkmaz/projects/saturn/hopt/hpopt_fa7_zinc/ray_results/TorchTrainer_2025-03-12_00-32-19/9c3b6b9f/checkpoint_000018)


Epoch 19: 100%|██████████| 28/28 [00:09<00:00,  3.10it/s, v_num=3.41e+7, train_loss_step=0.197, val_loss=0.168, train_loss_epoch=0.193]
Validation: |          | 0/? [00:00<?, ?it/s]
(RayTrainWorker pid=2734849) 
Validation DataLoader 0:   0%|          | 0/6 [00:00<?, ?it/s]
(RayTrainWorker pid=2734849) 
Validation DataLoader 0:  17%|█▋        | 1/6 [00:00<00:01,  4.90it/s]
(RayTrainWorker pid=2734849) 
Validation DataLoader 0:  33%|███▎      | 2/6 [00:00<00:00,  4.37it/s]
(RayTrainWorker pid=2734849) 
Validation DataLoader 0:  50%|█████     | 3/6 [00:00<00:00,  4.20it/s]
(RayTrainWorker pid=2734849) 
Validation DataLoader 0:  67%|██████▋   | 4/6 [00:00<00:00,  4.16it/s]
(RayTrainWorker pid=2734849) 
Validation DataLoader 0:  83%|████████▎ | 5/6 [00:01<00:00,  4.16it/s]
(RayTrainWorker pid=2734849) 
Epoch 19: 100%|██████████| 28/28 [00:10<00:00,  2.69it/s, v_num=3.41e+7, train_loss_step=0.197, val_loss=0.147, train_loss_epoch=0.193]


2025-03-12 00:37:22,230	WARNING experiment_state.py:206 -- Experiment state snapshotting has been triggered multiple times in the last 5.0 seconds and may become a bottleneck. A snapshot is forced if `CheckpointConfig(num_to_keep)` is set, and a trial has checkpointed >= `num_to_keep` times since the last snapshot.
You may want to consider increasing the `CheckpointConfig(num_to_keep)` or decreasing the frequency of saving checkpoints.
You can suppress this warning by setting the environment variable TUNE_WARN_EXCESSIVE_EXPERIMENT_CHECKPOINT_SYNC_THRESHOLD_S to a smaller value than the current threshold (5.0). Set it to 0 to completely suppress this warning.
(RayTrainWorker pid=2734849) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/ictstr01/home/aih/serra.korkmaz/projects/saturn/hopt/hpopt_fa7_zinc/ray_results/TorchTrainer_2025-03-12_00-32-19/9c3b6b9f/checkpoint_000019)
(RayTrainWorker pid=2734849) `Trainer.fit` stopped: `max_epochs=20` reached.


Epoch 19: 100%|██████████| 28/28 [00:10<00:00,  2.60it/s, v_num=3.41e+7, train_loss_step=0.197, val_loss=0.147, train_loss_epoch=0.178]


2025-03-12 00:37:23,496	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/ictstr01/home/aih/serra.korkmaz/projects/saturn/hopt/hpopt_fa7_zinc/ray_results/TorchTrainer_2025-03-12_00-32-19' in 0.0091s.
2025-03-12 00:37:23,502	INFO tune.py:1041 -- Total run time: 303.68 seconds (303.61 seconds for the tuning loop).


In [9]:
results

ResultGrid<[
  Result(
    metrics={'train_loss': 0.17810241878032684, 'train_loss_step': 0.19707699120044708, 'val/rmse': 0.3827739953994751, 'val/mae': 0.2959623336791992, 'val_loss': 0.14651592075824738, 'train_loss_epoch': 0.17810241878032684, 'epoch': 19, 'step': 560},
    path='/ictstr01/home/aih/serra.korkmaz/projects/saturn/hopt/hpopt_fa7_zinc/ray_results/TorchTrainer_2025-03-12_00-32-19/9c3b6b9f',
    filesystem='local',
    checkpoint=Checkpoint(filesystem=local, path=/ictstr01/home/aih/serra.korkmaz/projects/saturn/hopt/hpopt_fa7_zinc/ray_results/TorchTrainer_2025-03-12_00-32-19/9c3b6b9f/checkpoint_000019)
  ),
  Result(
    metrics={'train_loss': 0.19567809998989105, 'train_loss_step': 0.20320464670658112, 'val/rmse': 0.4064917266368866, 'val/mae': 0.3187048137187958, 'val_loss': 0.16523553431034088, 'train_loss_epoch': 0.19567809998989105, 'epoch': 19, 'step': 560},
    path='/ictstr01/home/aih/serra.korkmaz/projects/saturn/hopt/hpopt_fa7_zinc/ray_results/TorchTrainer_2025

In [10]:
result_df = results.get_dataframe()
result_df

,train_loss,train_loss_step,val/rmse,val/mae,val_loss,train_loss_epoch,epoch,step,timestamp,checkpoint_dir_name,...,pid,hostname,node_ip,time_since_restore,iterations_since_restore,config/train_loop_config/depth,config/train_loop_config/ffn_hidden_dim,config/train_loop_config/ffn_num_layers,config/train_loop_config/message_hidden_dim,logdir
0,0.178102,0.197077,0.382774,0.295962,0.146516,0.178102,19,560,1741736242,checkpoint_000019,...,2734610,cpusrv45.scidom.de,10.233.0.55,291.300657,20,2,2000,2,500,9c3b6b9f
1,0.195678,0.203205,0.406492,0.318705,0.165236,0.195678,19,560,1741736223,checkpoint_000019,...,2734850,cpusrv45.scidom.de,10.233.0.55,260.440540,20,2,2200,2,400,5a7e0717


In [11]:
# best configuration
best_result = results.get_best_result()
best_config = best_result.config
best_config['train_loop_config']

{'depth': 2,
 'ffn_hidden_dim': 2000,
 'ffn_num_layers': 2,
 'message_hidden_dim': 500}

In [12]:
# best model checkpoint path
best_result = results.get_best_result()
best_checkpoint_path = Path(best_result.checkpoint.path) / "checkpoint.ckpt"
print(f"Best model checkpoint path: {best_checkpoint_path}")

Best model checkpoint path: /ictstr01/home/aih/serra.korkmaz/projects/saturn/hopt/hpopt_fa7_zinc/ray_results/TorchTrainer_2025-03-12_00-32-19/9c3b6b9f/checkpoint_000019/checkpoint.ckpt


In [13]:
mpnn = models.MPNN.load_from_checkpoint(best_checkpoint_path)
mpnn

MPNN(
  (message_passing): BondMessagePassing(
    (W_i): Linear(in_features=86, out_features=500, bias=False)
    (W_h): Linear(in_features=500, out_features=500, bias=False)
    (W_o): Linear(in_features=572, out_features=500, bias=True)
    (dropout): Dropout(p=0.0, inplace=False)
    (tau): ReLU()
    (V_d_transform): Identity()
    (graph_transform): Identity()
  )
  (agg): MeanAggregation()
  (bn): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (predictor): RegressionFFN(
    (ffn): MLP(
      (0): Sequential(
        (0): Linear(in_features=500, out_features=2000, bias=True)
      )
      (1): Sequential(
        (0): ReLU()
        (1): Dropout(p=0.0, inplace=False)
        (2): Linear(in_features=2000, out_features=2000, bias=True)
      )
      (2): Sequential(
        (0): ReLU()
        (1): Dropout(p=0.0, inplace=False)
        (2): Linear(in_features=2000, out_features=1, bias=True)
      )
    )
    (criterion): MSE(task_weights=[[1.0]

In [14]:
import torch
test_loader = data.build_dataloader(test_dset, shuffle=False)
with torch.inference_mode():
    trainer = pl.Trainer(
        logger=None,
        enable_progress_bar=True,
        accelerator="cpu",
        devices=1
    )
    test_preds = trainer.predict(mpnn, test_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
SLURM auto-requeueing enabled. Setting signal handlers.
/home/aih/serra.korkmaz/miniconda3/envs/saturn/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.


Predicting DataLoader 0: 100%|██████████| 4/4 [00:00<00:00,  9.20it/s]


In [15]:
import numpy as np
test_preds = np.concatenate(test_preds, axis=0)
df_test['preds'] = test_preds
df_test

,smiles,value,preds
0,O=C1N[C@@]2(CCOc3ccccc32)C(=O)N1Cc1cccc(C(F)(F...,7.9,7.480560
1,Cc1nnc(-c2cc(CC(C)C)n(-c3cccc(C(=O)N[C@@H]4C[C...,7.7,7.195705
2,O=[N+]([O-])c1ccc(N(C[C@@H]2CCCO2)C[C@@H]2CCCO...,6.4,6.632491
3,CCOC(=O)CN1C(=O)S/C(=C\c2ccc(-c3ccc(Br)cc3)o2)...,5.9,6.426285
4,O=C(NC[C@@H]1CN(Cc2ccccc2)CCO1)C1=NN(c2ccccc2)...,7.9,7.288399
...,...,...,...
193,CC[C@@H](O)C[NH+]1CCC(NC(=O)[C@H](c2ccccc2)C(C...,6.2,5.890581
194,Cc1ccc([C@@H](CNc2nc3ccccn3c(=O)c2[N+](=O)[O-]...,8.3,7.432421
195,[NH-]C1=NC2=NC=N[C@H]2C(N2CCC(n3cc(-c4cccnc4)n...,8.3,8.579091
196,O=C([O-])C1CCN(C(=O)CCN2C(=O)[C@H]3Cc4ccccc4CN...,7.6,7.362548


In [16]:
from sklearn.metrics import r2_score, mean_squared_error

# Get true values (ground truth) and predictions
y_true = df_test['value'].values  # True target values
y_pred = df_test['preds'].values  # Predicted values

# Calculate metrics
r2 = r2_score(y_true, y_pred)
mse = mean_squared_error(y_true, y_pred)

print(f"R² Score: {r2:.4f}")
print(f"MSE: {mse:.4f}")

R² Score: 0.5821
MSE: 0.2693
